In [ ]:
import os
print(os.getcwd())
#os.chdir('../blocking/')

import pandas as pd
import py_entitymatching as em
import math
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import style
import re




In [ ]:
from numpy import genfromtxt
#myData = genfromtxt('train.csv',delimiter=',')
#myData
train = pd.read_csv('train_reduced.csv', encoding="ISO-8859-1", index_col='_id')
test = pd.read_csv('test_reduced.csv', encoding="ISO-8859-1", index_col='_id')

train = train.dropna()
test = test.dropna()


y = train['label']
cols = col for col in train.columns if col is not 'label'
X = train[cols]
X.head()

In [ ]:
#TODO use F-1 for scoring, or a hybrid if possible

# logistic regression
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
print('logistic regression:', cross_val_score(logreg, X, y, cv=10, scoring='accuracy').mean())

# decision tree


# random forest
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators=10, max_depth=10)
print('random forest:', cross_val_score(forest, X, y, cv=10, scoring='accuracy').mean())

# support vector machine
from sklearn.svm import LinearSVC
svm = LinearSVC()
print('SVM:', cross_val_score(svm, X, y, cv=10, scoring='accuracy').mean())

# naive bayes
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
print('Naive Bayes:', cross_val_score(svm, X, y, cv=10, scoring='accuracy').mean())

# KNN
knn = KNeighborsClassifier(n_neighbors=20)
print('KNN:', cross_val_score(knn, X, y, cv=10, scoring='accuracy').mean())
